In [2]:
!pip install haversine


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Thiago\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import io
import re
from haversine import haversine 

# Ler o arquivo CSV
df = pd.read_csv("C:/Users/Thiago/Documents/repos/ftc_programacao_python/train.csv")

# Remover linhas onde a idade do entregador é NaN
linhas_selecionadas = df["Delivery_person_Age"].notna()
df = df.loc[linhas_selecionadas, :].copy()

# Remover linhas onde a Cidade é em branco
linhas_selecionadas = (df["City"] !='NaN ')
df = df.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df["Festival"] !='NaN ')
df = df.loc[linhas_selecionadas, :].copy()

# Converter a idade do entregador para numérico, lidando com erros
df["Delivery_person_Age"] = pd.to_numeric(df["Delivery_person_Age"], errors='coerce')

# Remover linhas com NaN após conversão
df = df[df["Delivery_person_Age"].notna()]

# Converter a idade do entregador para inteiro
df["Delivery_person_Age"] = df["Delivery_person_Age"].astype(int)

# Converter as classificações dos entregadores para float, lidando com erros
df["Delivery_person_Ratings"] = pd.to_numeric(df['Delivery_person_Ratings'], errors='coerce')
df = df[df["Delivery_person_Ratings"].notna()]  # Remover linhas com NaN

# Converter a data do pedido para datetime
df["Order_Date"] = pd.to_datetime(df['Order_Date'], format='%d-%m-%Y')

# Remover linhas onde o número de entregas múltiplas é NaN
linhas_selecionadas = df['multiple_deliveries'].notna()
df = df.loc[linhas_selecionadas, :].copy()

# Converter múltiplas entregas para inteiro, lidando com erros
df['multiple_deliveries'] = pd.to_numeric(df['multiple_deliveries'], errors='coerce')
df = df[df['multiple_deliveries'].notna()]  # Remover linhas com NaN

# Converter múltiplas entregas para inteiro
df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)

df.loc[:, 'ID'] = df.loc[:, 'ID'].str.strip()
df.loc[:, 'Road_traffic_density'] = df.loc[:, 'Road_traffic_density'].str.strip()
df.loc[:, 'Type_of_order'] = df.loc[:, 'Type_of_order'].str.strip()
df.loc[:, 'Type_of_vehicle'] = df.loc[:, 'Type_of_vehicle'].str.strip()
df.loc[:, 'City'] = df.loc[:, 'City'].str.strip()
df.loc[:, 'Festival'] = df.loc[:, 'Festival'].str.strip()

#limpando a coluna time taken

df['Time_taken(min)'] = df['Time_taken(min)'].apply(lambda x: x.split('(min)')[1])
df['Time_taken(min)'] = df['Time_taken(min)'].astype(int)


# Exibir a forma do DataFrame
print(df.shape)

(41457, 20)


In [4]:
!pip install folium


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Thiago\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
#1.Quantidade de pedidos por dia

cols = ['ID', 'Order_Date']

df_aux = df.loc[:, cols].groupby('Order_Date').count().reset_index()

import plotly.express as px

px.bar( df_aux, x= 'Order_Date', y='ID')

In [5]:
#2 quantidade de pedidos de semana

df['week_of_year'] = df['Order_Date'].dt.strftime( '%U' )

df_aux = df.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

px.line(df_aux, x='week_of_year', y='ID')

NameError: name 'df' is not defined

In [38]:
#3 distribuição por tipo de grafico

df_aux = df.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()

df_aux = df_aux.loc[df_aux['Road_traffic_density'] != "NaN", :]
df_aux['entregas_perc'] =df_aux['ID']/df_aux['ID'].sum()

px.pie(df_aux, values='entregas_perc', names='Road_traffic_density')


In [39]:
df_aux = df.loc[:,['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
df_aux  = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux  = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City')

In [7]:
#5 A quantidade de pedidos por entregador por semana

df['Order_Date'] = pd.to_datetime(df['Order_Date'])

# Criar a coluna 'week_of_year' (semana do ano)
df['week_of_year'] = df['Order_Date'].dt.strftime('%U')  # %U retorna o número da semana do ano (domingo)

# Verificar se a coluna 'week_of_year' foi criada corretamente
print(df[['Order_Date', 'week_of_year']].head())

# 1. Número de pedidos por semana (contagem de 'ID')
df_aux_1 = df.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

# 2. Número único de entregadores por semana
df_aux_2 = df.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

# 3. Juntar os DataFrames com o merge
df_aux = pd.merge(df_aux_1, df_aux_2, on='week_of_year', how='inner')

# 4. Calcular a quantidade média de pedidos por entregador por semana
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']

# 5. Criar o gráfico de linha
fig = px.line(df_aux, x='week_of_year', y='order_by_deliver', title='Pedidos por Entregador por Semana')
fig.show()

  Order_Date week_of_year
0 2022-03-19           11
1 2022-03-25           12
2 2022-03-19           11
3 2022-04-05           14
4 2022-03-26           12


In [4]:
import folium

df_aux = df.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()
df_aux = df_aux.loc[df_aux['City']!= 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density']!= 'NaN', :]

map = folium.Map()

for index, location_info in df_aux.iterrows():
    folium.Marker([location_info['Delivery_location_latitude'], 
                 location_info['Delivery_location_longitude']]).add_to(map)

map

In [8]:
#1. A menor e maior idade dos entregadores

#maior idade

print('A maior idade eh: {}' .format (df.loc[:, 'Delivery_person_Age'].max()))
print('A maior idade eh: {}' .format (df.loc[:, 'Delivery_person_Age'].min()))

A maior idade eh: 50
A maior idade eh: 15


In [9]:
#2. A pior e a melhor condição de veículos

print('A melhor condição de veículo eh: {}' .format (df.loc[:, 'Vehicle_condition'].max()))
print('A pior condição de veículo eh: {}' .format (df.loc[:, 'Vehicle_condition'].min()))


A melhor condição de veículo eh: 3
A pior condição de veículo eh: 0


In [18]:
#3. A avaliação média por entregador.

df.loc[:,['Delivery_person_Ratings','Delivery_person_ID']].groupby('Delivery_person_ID').mean().reset_index()


,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.625862
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.626786
1318,VADRES20DEL02,4.593617


In [21]:
#4. A avaliação média e o desvio padrão por tipo de tráfego.

df.loc[:,['Delivery_person_Ratings','Road_traffic_density']].groupby('Road_traffic_density').agg({'mean','std'}).reset_index()

Road_traffic_density Delivery_person_Ratings          
                                           std      mean
0                 High                0.272312  4.653492
1                  Jam                0.328749  4.595395
2                  Low                0.335426  4.646780
3               Medium                0.273472  4.660627
4                  NaN                2.479464  3.912088

In [30]:
#5. A avaliação média e o desvio padrão por condições climáticas.

df_avg_std_rating_by_weather = (
    df.loc[:,['Delivery_person_Ratings','Weatherconditions']]
    .groupby('Weatherconditions')
    .agg( delivery_mean = ('Delivery_person_Ratings','mean'), delivery_std= ('Delivery_person_Ratings', 'std')).reset_index())


df_avg_std_rating_by_weather

,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.652190,0.280269
1,conditions Fog,4.653549,0.273893
2,conditions NaN,3.912088,2.479464
3,conditions Sandstorms,4.613000,0.309512
4,conditions Stormy,4.613369,0.311624
5,conditions Sunny,4.658030,0.393090
6,conditions Windy,4.617804,0.303464


In [44]:
#6 Os 10 entregadores mais rápidos por cidade

df.loc[:,['Delivery_person_ID','Time_taken(min)','City']].groupby(['City','Delivery_person_ID']).min().sort_values(['City','Time_taken(min)']).reset_index()



,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES010DEL03,10
1,Metropolitian,AGRRES07DEL03,10
2,Metropolitian,AGRRES12DEL01,10
3,Metropolitian,AGRRES14DEL01,10
4,Metropolitian,AGRRES17DEL03,10
...,...,...,...
3397,Urban,DEHRES02DEL03,41
3398,Urban,BHPRES03DEL02,42
3399,Urban,AURGRES17DEL03,43
3400,Urban,LUDHRES16DEL03,47


In [5]:
#1. A quantidade de entregadores únicos.
df.loc[:, "Delivery_person_ID"].nunique()

1320

In [ ]:
#2. A distância média dos resturantes e dos locais de entrega.

cols = ['Restaurant_latitude','Restaurant_longitude', 'Delivery_location_latitude','Delivery_location_longitude']

df['distance'] = df.loc[:,cols].apply(lambda x: haversine( (x['Restaurant_latitude'], x['Restaurant_longitude']),(x['Delivery_location_latitude'],x['Delivery_location_longitude'])) , axis=1)

df['distance'].mean()

fig = go.figure ( da)

np.float64(37.99189347335687)

In [7]:
#3. O tempo médio e o desvio padrão de entrega por cidade.

cols = ['City', 'Time_taken(min)']

df.loc[:, cols].groupby('City').agg( avg_time_delivery = ('Time_taken(min)','mean'), std_time_delivery = ('Time_taken(min)','std') ).reset_index()

,City,avg_time_delivery,std_time_delivery
0,Metropolitian,27.427304,9.132618
1,Semi-Urban,49.710526,2.724992
2,Urban,23.206179,8.852893


In [8]:
#4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.

cols = ['City', 'Time_taken(min)', 'Type_of_order']

df.loc[:, cols].groupby(['City','Type_of_order']).agg( avg_time_delivery = ('Time_taken(min)','mean'), std_time_delivery = ('Time_taken(min)','std') ).reset_index()


,City,Type_of_order,avg_time_delivery,std_time_delivery
0,Metropolitian,Buffet,27.308005,9.156311
1,Metropolitian,Drinks,27.317607,9.038826
2,Metropolitian,Meal,27.610103,9.216771
3,Metropolitian,Snack,27.467990,9.114290
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.565553,9.057550
9,Urban,Drinks,23.303766,8.921204


In [9]:
#5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego

cols = ['City', 'Time_taken(min)', 'Road_traffic_density']

df.loc[:, cols].groupby(['City','Road_traffic_density']).agg( avg_time_delivery = ('Time_taken(min)','mean'), std_time_delivery = ('Time_taken(min)','std') ).reset_index()

,City,Road_traffic_density,avg_time_delivery,std_time_delivery
0,Metropolitian,High,28.145976,7.904494
1,Metropolitian,Jam,31.978193,9.477798
2,Metropolitian,Low,22.259874,6.795802
3,Metropolitian,Medium,27.728045,8.309769
4,Metropolitian,NaN,26.156250,8.227362
5,Semi-Urban,High,50.125000,2.629956
6,Semi-Urban,Jam,49.841270,2.717095
7,Semi-Urban,Medium,47.400000,2.011080
8,Urban,High,24.305335,8.494842
9,Urban,Jam,27.989541,10.078679


In [10]:
#6. O tempo médio de entrega durantes os Festivais.

cols = ['Time_taken(min)', 'Festival']

df_aux = df.loc[:, cols].groupby('Festival').agg( avg_time_delivery = ('Time_taken(min)','mean'), std_time_delivery = ('Time_taken(min)','std') ).reset_index()

linhas_selecionadas = df_aux['Festival'] =='Yes'
df_aux = df_aux.reset_index()
df_aux = df_aux.loc[linhas_selecionadas,:]
df_aux

,index,Festival,avg_time_delivery,std_time_delivery
1,1,Yes,45.51976,4.001862
